In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [17]:
data = pd.read_csv('train.csv',encoding='latin1')
data.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [18]:
data = data[['text','sentiment']]

In [20]:
data = data[data.sentiment != "neutral"]
data['text'] = data['text'].apply(lambda x: x.lower() if isinstance(x, str) else x)
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x) if isinstance(x, str) else x))

print(data[ data['sentiment'] == 'positive'].size)
print(data[ data['sentiment'] == 'negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

17164
15562


In [21]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 32, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(10963, 32) (10963, 2)
(5400, 32) (5400, 2)


In [23]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
343/343 - 83s - loss: 0.4451 - accuracy: 0.7810 - 83s/epoch - 243ms/step
Epoch 2/7
343/343 - 77s - loss: 0.2858 - accuracy: 0.8812 - 77s/epoch - 224ms/step
Epoch 3/7
343/343 - 79s - loss: 0.2422 - accuracy: 0.9019 - 79s/epoch - 231ms/step
Epoch 4/7
343/343 - 79s - loss: 0.2154 - accuracy: 0.9164 - 79s/epoch - 230ms/step
Epoch 5/7
343/343 - 70s - loss: 0.1917 - accuracy: 0.9289 - 70s/epoch - 204ms/step
Epoch 6/7
343/343 - 71s - loss: 0.1687 - accuracy: 0.9366 - 71s/epoch - 207ms/step
Epoch 7/7
343/343 - 71s - loss: 0.1532 - accuracy: 0.9423 - 71s/epoch - 207ms/step


In [24]:
validation_size = 1500
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

122/122 - 4s - loss: 0.3848 - accuracy: 0.8710 - 4s/epoch - 37ms/step
score: 0.38
acc: 0.87


In [25]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 1s - 518ms/epoch - 518ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 34ms/epoch - 34ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 31ms/epoch - 31ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 33ms/epoch - 33ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 34ms/epoch - 34ms/step
1/1 - 0s - 34ms/epoch - 34ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 4

1/1 - 0s - 34ms/epoch - 34ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 34ms/epoch - 34ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 31ms/epoch - 31ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 54ms/epoch - 54ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 42m

1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 58ms/epoch - 58ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 34ms/epoch - 34ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 57ms/epoch - 57ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 57ms/epoch - 57ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 54ms/epoch - 54ms/step
1/1 - 0s - 34ms/epoch - 34ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 58ms/epoch - 58ms/step
1/1 - 0s - 54ms/epoch - 54ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 52m

1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 53ms/epoch - 53ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 60ms/epoch - 60ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 59ms/epoch - 59ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 59ms/epoch - 59ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 34ms/epoch - 34ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 55m

1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 54ms/epoch - 54ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 53ms/epoch - 53ms/step
1/1 - 0s - 79ms/epoch - 79ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 58ms/epoch - 58ms/step
1/1 - 0s - 57ms/epoch - 57ms/step
1/1 - 0s - 53ms/epoch - 53ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 69ms/epoch - 69ms/step
1/1 - 0s - 55ms/epoch - 55ms/step
1/1 - 0s - 53ms/epoch - 53ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 54ms/epoch - 54ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 52m

1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 66ms/epoch - 66ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 56ms/epoch - 56ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 56ms/epoch - 56ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 68ms/epoch - 68ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 64ms/epoch - 64ms/step
1/1 - 0s - 58ms/epoch - 58ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 51m

1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 55ms/epoch - 55ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 69ms/epoch - 69ms/step
1/1 - 0s - 61ms/epoch - 61ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 67ms/epoch - 67ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 69ms/epoch - 69ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 34ms/epoch - 34ms/step
1/1 - 0s - 50m

In [36]:
twt = ['I love humanity']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  1 32]]
1/1 - 0s - 56ms/epoch - 56ms/step
positive
